In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_train_x = mnist.train.images
mnist_train_y = mnist.train.labels
mnist_train_y = np.argmax(mnist_train_y, axis=1)

mnist_valid_x = mnist.validation.images
mnist_valid_y = mnist.validation.labels
mnist_valid_y = np.argmax(mnist_valid_y, axis=1)

print mnist_train_x.shape
print mnist_valid_x.shape

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(5000, 784)


In [3]:
from IPython.display import display, Image
import PIL.Image
from cStringIO import StringIO

import tensorflow as tf
import numpy as np

def display_digit(x):
    x = np.reshape(x, [28, 28])
    x = np.copy(x)
    x *= 256
    x = np.clip(x, 0., 255.)
    x = x.astype(np.uint8)
    
    f = StringIO()
    PIL.Image.fromarray(x).save(f, 'png')
    display(Image(data=f.getvalue()))

i = 3
display_digit(mnist_train_x[i])
print mnist_train_y[i]

6


In [12]:
lr = 0.01
eps = 0.01

import tensorflow as tf

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None])
adv = tf.placeholder(tf.bool, [])

with tf.variable_scope('adversary'):
    yoh = tf.one_hot(y, 10)
    xy = tf.concat([x, yoh], axis=1)
    
    with tf.variable_scope('h0'):
        xadv = tf.layers.dense(xy, 392)
    
    with tf.variable_scope('output'):
        xadv = tf.layers.dense(xadv, 784)
    
    xadv = eps * xadv

with tf.variable_scope('classifier'):
    xh = tf.cond(adv, lambda: x, lambda: x + xadv)
    
    with tf.variable_scope('h0'):
        xh = tf.layers.dense(x, 392)
        
    with tf.variable_scope('softmax'):
        xh = tf.layers.dense(xh, 10)

nll = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=xh))

preds = tf.argmax(xh, axis=1, output_type=tf.int32)
correct = tf.equal(preds, y)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

cls_opt = tf.train.GradientDescentOptimizer(lr).minimize(nll, global_step=tf.train.get_or_create_global_step())
adv_opt = tf.train.GradientDescentOptimizer(lr).minimize(-nll)

In [19]:
nepochs = 8
batch_size = 50
_adv = False

import random as random

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in xrange(nepochs):
    print 'Epoch {}'.format(i)
    
    print 'Validation accuracy: {}'.format(sess.run(accuracy, {x: mnist_valid_x, y: mnist_valid_y, adv: False}))
    
    _is = set(range(mnist_train_x.shape[0]))
    while len(_is) > batch_size:
        _i = random.sample(_is, batch_size)
        _is -= set(_i)
        _x = np.take(mnist_train_x, _i, axis=0)
        _y = np.take(mnist_train_y, _i, axis=0)
        
        if _adv:
            _ = sess.run(adv_opt, {x: _x, y: _y, adv: _adv})
        _ = sess.run(cls_opt, {x: _x, y: _y, adv: _adv})

Epoch 0
Validation accuracy: 0.097800001502
Epoch 1
Validation accuracy: 0.894400000572
Epoch 2
Validation accuracy: 0.907199978828
Epoch 3
Validation accuracy: 0.91219997406
Epoch 4
Validation accuracy: 0.916000008583
Epoch 5
Validation accuracy: 0.91939997673
Epoch 6
Validation accuracy: 0.920799970627
Epoch 7
Validation accuracy: 0.921599984169


In [20]:
saver = tf.train.Saver()
saver.save(sess, './reg', global_step=8)

'./reg-8'

In [81]:
eps = 0.25

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [784])
y = tf.placeholder(tf.int32, [])
yadv = tf.placeholder(tf.int32, [])

xe = tf.expand_dims(x, axis=0)
with tf.variable_scope('classifier'):
    xh = xe
    
    with tf.variable_scope('h0'):
        xh = tf.layers.dense(xh, 392)
        
    with tf.variable_scope('softmax'):
        xh = tf.layers.dense(xh, 10)
    
    scores = xh[0]

with tf.variable_scope('adv'):
    xadv = tf.get_variable('adv', [784], tf.float32, initializer=tf.random_normal_initializer)
xpert = xe + eps * tf.expand_dims(xadv, axis=0)
    
with tf.variable_scope('classifier', reuse=True):
    xh = xpert
    
    with tf.variable_scope('h0'):
        xh = tf.layers.dense(xh, 392)
        
    with tf.variable_scope('softmax'):
        xh = tf.layers.dense(xh, 10)
    
    scores_adv = xh[0]
cls_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='classifier')
    
nll_adv = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=yadv, logits=scores_adv))

preds = tf.nn.softmax(scores)
preds_adv = tf.nn.softmax(scores_adv)

adv_opt = tf.train.GradientDescentOptimizer(lr).minimize(nll_adv, var_list=[xadv])

saver = tf.train.Saver(cls_vars)

In [84]:
i = 10
nadv = 1000

#ckpt_fp = 'adv-135'
ckpt_fp = 'reg-8'

_x = mnist_valid_x[i]
_y = mnist_valid_y[i]
_yadv = 8

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, ckpt_fp)
    
    np.set_printoptions(suppress=True)
    
    _xpert, _preds, _preds_adv = sess.run([xpert, preds, preds_adv], {x: _x})
    display_digit(_x)
    print _preds
    
    display_digit(_xpert)
    print _preds_adv
    
    _preds = sess.run([preds], {x: 1. - _x})
    display_digit(1. - _x)
    print _preds
    
    for i in xrange(nadv):
        sess.run(adv_opt, {x: _x, yadv: _yadv})
        
    _xpert, _preds, _preds_adv = sess.run([xpert, preds, preds_adv], {x: _x})
    display_digit(_x)
    print _preds
    
    display_digit(_xpert)
    print _preds_adv

INFO:tensorflow:Restoring parameters from reg-8


[ 0.00000806  0.0000909   0.00024036  0.99723971  0.00000011  0.00028115
  0.00000012  0.00000687  0.0016862   0.00044652]


[ 0.00000365  0.00002292  0.00030532  0.99676251  0.00000018  0.00031764
  0.00000009  0.00000453  0.00238169  0.00020133]


[array([ 0.00648754,  0.        ,  0.20214675,  0.00001952,  0.00000001,
        0.79105622,  0.00000244,  0.00000018,  0.0002873 ,  0.        ], dtype=float32)]


[ 0.00000806  0.0000909   0.00024036  0.99723971  0.00000011  0.00028115
  0.00000012  0.00000687  0.0016862   0.00044652]


[ 0.0000673   0.00023437  0.00096288  0.09448223  0.00001231  0.00636823
  0.00000707  0.00000759  0.89366025  0.00419784]


* Regular @ (eps=0.25, nadv=1000): [99.7%, 0.1%] -> [11.3%, 86.4%]
* Adv @ (eps=0.25, nadv=1000): [90.4%, 0.2%] -> [64.3%, 15.1%]